#EasyOCR

In [5]:
!pip3 install easyocr --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
import os, zipfile, easyocr
from PIL import Image
from tqdm import tqdm
from google.colab import drive

In [7]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/part_1.zip'
if not os.path.isdir('/content/data'):
  os.mkdir('/content/data')
os.chdir('/content/data')
os.path.abspath('.')
if not os.path.isdir('/content/data/part_1'):
  if os.path.exists(zip_path):
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall('./')
  else:
      print("Архив не найден по пути:", zip_path)
df = pd.read_csv("./part_1/all_files_final.csv")
df = df.dropna()
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,new_path,word_true
0,0,./part_1/b/baulina_0.jpg,баулина
1,1,./part_1/n/natal'ja_1.jpg,наталья
2,2,./part_1/b/biblioteka_2.jpg,библиотека
3,3,./part_1/f/frolova_3.jpg,фролова
4,4,./part_1/o/ol'ga_4.jpg,ольга


In [14]:
# Инициализация EasyOCR
reader = easyocr.Reader(['ru'], gpu=True)  # Используй gpu=False, если нет CUDA


In [40]:
import re

results = []


for i, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row['new_path']
    word_true = str(row['word_true']).strip().lower()

    try:
        result = reader.readtext(img_path, detail=0, paragraph=False, decoder='greedy')
        prediction = re.sub(r'[^а-яё]', '', result[0].lower(), flags=re.IGNORECASE) if result else ""
    except Exception as e:
        prediction = ""

    results.append({
        "id": row["id"],
        "true": word_true,
        "pred": prediction,
        "match": word_true == prediction
    })

df_results = pd.DataFrame(results)
df_results.to_csv("easyocr_results.csv", index=False)


100%|██████████| 24591/24591 [16:39<00:00, 24.60it/s]


In [41]:
accuracy = df_results["match"].mean()
print(f"🔍 Точность EasyOCR: {accuracy:.2%}")

# Несовпадения
df_results[df_results["match"] == False].head(1000)


🔍 Точность EasyOCR: 3.12%


,id,true,pred,match
0,0,баулина,ауин,False
1,1,наталья,нааья,False
2,2,библиотека,иотка,False
3,3,фролова,фв,False
4,4,ольга,,False
...,...,...,...,...
1024,1024,александра,ш,False
1025,1025,лженосов,е,False
1026,1026,балдина,и,False
1027,1027,мария,м,False


#TesseractOCR

In [1]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-rus
!pip install pytesseract

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,932 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packa

In [2]:
import os
import pandas as pd
import pytesseract
from PIL import Image
from tqdm import tqdm
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/part_1.zip'
if not os.path.isdir('/content/data'):
  os.mkdir('/content/data')
os.chdir('/content/data')
os.path.abspath('.')
if not os.path.isdir('/content/data/part_1'):
  if os.path.exists(zip_path):
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall('./')
  else:
      print("Архив не найден по пути:", zip_path)
df = pd.read_csv("./part_1/all_files_final.csv")
df = df.dropna()
df.head()

Mounted at /content/drive
✅ Загружено 24591 строк.


,id,new_path,word_true
0,0,./part_1/b/baulina_0.jpg,баулина
1,1,./part_1/n/natal'ja_1.jpg,наталья
2,2,./part_1/b/biblioteka_2.jpg,библиотека
3,3,./part_1/f/frolova_3.jpg,фролова
4,4,./part_1/o/ol'ga_4.jpg,ольга


In [3]:
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

results = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row['new_path']
    word_true = str(row['word_true']).strip().lower()

    try:
        img = Image.open(img_path)
        prediction = pytesseract.image_to_string(img, lang='rus', config='--psm 7')
        prediction = prediction.strip().lower()
    except Exception as e:
        prediction = ""

    results.append({
        "id": row["id"],
        "true": word_true,
        "pred": prediction,
        "match": word_true == prediction
    })

df_results = pd.DataFrame(results)
df_results.to_csv("tesseract_results.csv", index=False)


100%|██████████| 24591/24591 [39:49<00:00, 10.29it/s]


In [4]:
accuracy = df_results["match"].mean()
print(f"🔍 Точность Tesseract OCR: {accuracy:.2%}")

# Несовпадения
df_results[df_results["match"] == False].head(10)


🔍 Точность Tesseract OCR: 20.34%


,id,true,pred,match
0,0,баулина,бахлина,False
2,2,библиотека,иен,False
3,3,фролова,ее,False
4,4,ольга,голови вар,False
5,5,манго,мааккто,False
7,7,юлия,в,False
8,8,юлюшка,а,False
9,9,гревцева,00232.22,False
12,12,клёсова,клесова,False
13,13,алиса,гак,False
